In [25]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [26]:
# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("MongoDB and Postgres") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.2,org.postgresql:postgresql:42.7.4") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/dbmycrawler") \
    .getOrCreate()

In [27]:
#doc du lieu tu mongo
df=spark.read.format("mongo").option("collection","tblcafeland").load()
#hien du lieu
df.show() 

+--------------------+--------------+------------------+--------------------+----------+-----------+--------------------+-------------------------+--------------------+--------+--------------------+--------------------+
|          Chủ đầu tư|Diện tích (m²)|        Loại dự án|               Mô tả| Ngày đăng| Trạng thái|           Tên dự án|Tổng vốn đầu tư (tỷ đồng)|              Vị trí|Xếp hạng|                 _id|               Đường|
+--------------------+--------------+------------------+--------------------+----------+-----------+--------------------+-------------------------+--------------------+--------+--------------------+--------------------+
|Công Ty Cổ Phần B...|        8200.0|     Đất Nền Dự Án|The Long Eyes là ...|2024-09-12|Đang mở bán|The Long Eyes: Dự...|                   275.46|Trảng Bom - Đồng Nai|     4.0|{6740756bc13e184e...|   đường Sông Thao 5|
|Công ty Cổ phần Đ...|       33540.7|   Căn hộ chung cư|Golden City là că...|2024-09-25|Đang mở bán|Golden City: Dự á...

In [28]:
#Xem kiểu dl các cột
df.printSchema()

root
 |-- Chủ đầu tư: string (nullable = true)
 |-- Diện tích (m²): double (nullable = true)
 |-- Loại dự án: string (nullable = true)
 |-- Mô tả: string (nullable = true)
 |-- Ngày đăng: string (nullable = true)
 |-- Trạng thái: string (nullable = true)
 |-- Tên dự án: string (nullable = true)
 |-- Tổng vốn đầu tư (tỷ đồng): double (nullable = true)
 |-- Vị trí: string (nullable = true)
 |-- Xếp hạng: double (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- Đường: string (nullable = true)



In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col
#Xóa giá trị "unknown" và null trong tất cả các cột của DataFrame, bạn có thể làm như sau:
columns = df.columns
for column in columns:
    df = df.filter(
        (col(column).isNotNull()) & (col(column) != "unknown")
    )
df.show()


In [29]:
from pyspark.sql.functions import col, regexp_replace
# Loại bỏ chuỗi 'Cập nhật:' trong cột 'Ngày đăng'
df = df.withColumn("Ngày đăng", regexp_replace(col("Ngày đăng"), "Cập nhật:", ""))

In [30]:
#Xem dl các cột
df.show()

+--------------------+--------------+------------------+--------------------+----------+-----------+--------------------+-------------------------+--------------------+--------+--------------------+--------------------+
|          Chủ đầu tư|Diện tích (m²)|        Loại dự án|               Mô tả| Ngày đăng| Trạng thái|           Tên dự án|Tổng vốn đầu tư (tỷ đồng)|              Vị trí|Xếp hạng|                 _id|               Đường|
+--------------------+--------------+------------------+--------------------+----------+-----------+--------------------+-------------------------+--------------------+--------+--------------------+--------------------+
|Công Ty Cổ Phần B...|        8200.0|     Đất Nền Dự Án|The Long Eyes là ...|2024-09-12|Đang mở bán|The Long Eyes: Dự...|                   275.46|Trảng Bom - Đồng Nai|     4.0|{6740756bc13e184e...|   đường Sông Thao 5|
|Công ty Cổ phần Đ...|       33540.7|   Căn hộ chung cư|Golden City là că...|2024-09-25|Đang mở bán|Golden City: Dự á...

In [ ]:
# Xóa bỏ dấu chấm ở các dòng m2
df = df.withColumn("Diện tích:", regexp_replace(col("Diện tích:"), "\\.", ""))

# Đổi dấu phẩy ở dòng có chứa 'ha' thành dấu chấm
df = df.withColumn("Diện tích:", regexp_replace(col("Diện tích:"), ",", "."))

In [ ]:
from pyspark.sql.functions import col, when, regexp_extract
# Chuyển đổi giá trị diện tích, nhân với 10.000 nếu có 'ha'
df = df.withColumn(
    "Diện tích:",
        when(
            col("Diện tích:").contains("ha"),
            regexp_extract(col("Diện tích:"), "([\\d.]+)", 1).cast("float") * 10000  # Nhân với 10.000 nếu có 'ha'
        ).otherwise(
            regexp_extract(col("Diện tích:"), "([\\d.]+)", 1).cast("float")  # Chuyển đổi giá trị m2
        )
)
# Đổi tên cột thành "Diện tích(m²)"
df = df.withColumnRenamed("Diện tích:", "Diện tích (m²)")
# Xóa bỏ chữ 'ha' và 'm2'
df = df.withColumn("Diện tích (m²)", regexp_replace(col("Diện tích (m²)"), "ha|m2", ""))
#Xem dl các cột
df.show()

+--------------------+--------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+--------+--------------------+--------------------+
|          Chủ đầu tư|Diện tích (m²)|        Loại dự án|               Mô tả|           Ngày đăng|           Thành phố| Trạng thái|           Tên dự án|  Tổng vốn đầu tư|Xếp hạng|                 _id|               Đường|
+--------------------+--------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+--------+--------------------+--------------------+
|Công Ty Cổ Phần B...|        8200.0|     Đất Nền Dự Án|The Long Eyes là ...|  12/09/2024 8:49 AM|           Trảng Bom|Đang mở bán|The Long Eyes: Dự...|    chưa cập nhật|     5.0|{66f8f9c98ac17558...|   đường Sông Thao 5|
|Công ty Cổ phần Đ...|       33540.7|   Căn hộ chung cư|Golden City là că...|  25/09/2024 9:05 AM|        TP. Tâ

In [ ]:
# Xóa bỏ dấu chấm ở các dòng "tỷ đồng"
df = df.withColumn("Tổng vốn đầu tư", regexp_replace(col("Tổng vốn đầu tư"), "\\.", ""))

# Đổi dấu phẩy ở dòng có chứa 'USD' thành dấu chấm
df = df.withColumn("Tổng vốn đầu tư", regexp_replace(col("Tổng vốn đầu tư"), ",", "."))

In [ ]:
from pyspark.sql.functions import col, when, regexp_extract
# Chuyển đổi giá trị tổng vón đầu tư, nhân với 24 nếu có "USD"
df = df.withColumn(
    "Tổng vốn đầu tư",
        when(
            col("Tổng vốn đầu tư").contains("USD"),
            regexp_extract(col("Tổng vốn đầu tư"), "([\\d.]+)", 1).cast("float") * 24  # Nhân 24 nếu có chữ triệu USD
        ).otherwise(
            regexp_extract(col("Tổng vốn đầu tư"), "([\\d.]+)", 1).cast("float")  
        )
)
# Đổi tên cột thành "Tổng vốn đầu tư (tỷ đồng)"
df = df.withColumnRenamed("Tổng vốn đầu tư", "Tổng vốn đầu tư (tỷ đồng)")

#đổi tên cột thành phố
df=df.withColumnRenamed("Thành phố",'Vị trí')

# Hiển thị dữ liệu sau khi thay đổi
df.show()

+--------------------+--------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------------------------+--------+--------------------+--------------------+
|          Chủ đầu tư|Diện tích (m²)|        Loại dự án|               Mô tả|           Ngày đăng|              Vị trí| Trạng thái|           Tên dự án|Tổng vốn đầu tư (tỷ đồng)|Xếp hạng|                 _id|               Đường|
+--------------------+--------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------------------------+--------+--------------------+--------------------+
|Công Ty Cổ Phần B...|        8200.0|     Đất Nền Dự Án|The Long Eyes là ...|  12/09/2024 8:49 AM|           Trảng Bom|Đang mở bán|The Long Eyes: Dự...|            chưa cập nhật|     5.0|{66f8f9c98ac17558...|   đường Sông Thao 5|
|Công ty Cổ phần Đ...|       33540.7|   Căn hộ chung cư|Golden City là că...|  2

In [31]:
from pyspark.sql.functions import col

# Danh sách cột mong muốn
desired_columns = [
    "Loại dự án",
    "Tên dự án",
    "Đường",
    "Vị trí",
    "Chủ đầu tư",
    "Trạng thái",
    "Diện tích (m²)",  # Đổi tên cột này cho đầy đủ
    "Ngày đăng",
    "Tổng vốn đầu tư (tỷ đồng)",  # Đổi tên cột này cho đầy đủ
    "Xếp hạng",
    "Mô tả"
]

# Kiểm tra danh sách các cột có trong DataFrame
existing_columns = df.columns
print("Các cột hiện có trong DataFrame:", existing_columns)

# Lọc ra các cột tồn tại trong DataFrame
valid_columns = [col for col in desired_columns if col in existing_columns]

# Sắp xếp lại DataFrame theo thứ tự các cột đã lọc
df = df.select(*valid_columns)

# Hiển thị dữ liệu sau khi thay đổi
df.show()


Các cột hiện có trong DataFrame: ['Chủ đầu tư', 'Diện tích (m²)', 'Loại dự án', 'Mô tả', 'Ngày đăng', 'Trạng thái', 'Tên dự án', 'Tổng vốn đầu tư (tỷ đồng)', 'Vị trí', 'Xếp hạng', '_id', 'Đường']
+------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------+----------+-------------------------+--------+--------------------+
|        Loại dự án|           Tên dự án|               Đường|              Vị trí|          Chủ đầu tư| Trạng thái|Diện tích (m²)| Ngày đăng|Tổng vốn đầu tư (tỷ đồng)|Xếp hạng|               Mô tả|
+------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------+----------+-------------------------+--------+--------------------+
|     Đất Nền Dự Án|The Long Eyes: Dự...|   đường Sông Thao 5|Trảng Bom - Đồng Nai|Công Ty Cổ Phần B...|Đang mở bán|        8200.0|2024-09-12|                   275.46|     4.0|The Long Eyes là ...|
|   Căn 

In [32]:
from pyspark.sql.functions import when
from pyspark.sql.functions import monotonically_increasing_id

#thêm cột project_id 
df=df.withColumn('project_id',monotonically_increasing_id())

# Thêm cột project_type_id với giá trị tương ứng cho từng loại dự án
df= df.withColumn("project_type_id",
    when(df["Loại dự án"] == "Khu công nghiệp", 111)
    .when(df["Loại dự án"] == "Nhà Phố - Biệt Thự", 112)
    .when(df["Loại dự án"] == "Bất Động Sản Nghỉ Dưỡng", 113)
    .when(df["Loại dự án"] == "Loại Hình Khác", 114)
    .when(df["Loại dự án"] == "Căn hộ chung cư", 115)
    .when(df["Loại dự án"] == "Khu Đô Thị", 116)
    .when(df["Loại dự án"] == "Đất Nền Dự Án", 117)
    .otherwise(None)  # Gán None cho các giá trị không xác định
)

In [33]:
from pyspark.sql import SparkSession
import shutil
import os

# Tạo SparkSession
spark = SparkSession.builder.appName("LuuCSV").getOrCreate()

# Ghi dữ liệu vào thư mục (chỉ một file)
output_dir = "dulieusach_output"
df.coalesce(1).write.csv(output_dir, header=True, mode='overwrite')

# Di chuyển file CSV duy nhất ra ngoài và xóa thư mục
for filename in os.listdir(output_dir):
    if filename.endswith(".csv"):
        shutil.move(os.path.join(output_dir, filename), "dulieusach.csv")

# Xóa thư mục trống sau khi di chuyển
shutil.rmtree(output_dir)

print("Dữ liệu đã được lưu thành công vào file dulieusach.csv")


Dữ liệu đã được lưu thành công vào file dulieusach.csv


In [34]:
df.show()

+------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------+----------+-------------------------+--------+--------------------+----------+---------------+
|        Loại dự án|           Tên dự án|               Đường|              Vị trí|          Chủ đầu tư| Trạng thái|Diện tích (m²)| Ngày đăng|Tổng vốn đầu tư (tỷ đồng)|Xếp hạng|               Mô tả|project_id|project_type_id|
+------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------+----------+-------------------------+--------+--------------------+----------+---------------+
|     Đất Nền Dự Án|The Long Eyes: Dự...|   đường Sông Thao 5|Trảng Bom - Đồng Nai|Công Ty Cổ Phần B...|Đang mở bán|        8200.0|2024-09-12|                   275.46|     4.0|The Long Eyes là ...|         0|            117|
|   Căn hộ chung cư|Golden City: Dự á...|số 6 đường Hồ Văn...|TP. Tây Ninh - Tâ...|Công ty Cổ ph

In [35]:
#bảng project
projects=df.select("Tên dự án",'project_type_id') \

    #thêm cột project_id để làm khóa chính và đổi tên các cột
projects=projects.withColumn('project_id',monotonically_increasing_id())\
                 .withColumnRenamed('project_id','Mã dự án') \
                 .withColumnRenamed('project_type_id','Mã loại dự án')

#bảng loại dự án
project_types=df.select("Loại dự án")
    #tạo khóa chính mã loại dự án
project_types= project_types.withColumn("project_type_id",
                                        when(col("Loại dự án") == "Khu công nghiệp", 111)
                                        .when(col("Loại dự án") == "Nhà Phố - Biệt Thự", 112)
                                        .when(col("Loại dự án") == "Bất Động Sản Nghỉ Dưỡng", 113)
                                        .when(col("Loại dự án") == "Loại Hình Khác", 114)
                                        .when(col("Loại dự án") == "Căn hộ chung cư", 115)
                                        .when(col("Loại dự án") == "Khu Đô Thị", 116)
                                        .when(col("Loại dự án") == "Đất Nền Dự Án", 117)
                                        .otherwise(None))

from pyspark.sql.functions import count

    # Tính số lượng dự án theo loại dự án
project_types = project_types.groupBy("project_type_id", "Loại dự án").agg(
    count("*").alias("Số lượng dự án")
)  
                
    # Loại bỏ các hàng trùng lặp
project_types = project_types.dropDuplicates(["project_type_id","Loại dự án","Số lượng dự án"])   

    #đổi tên các cột
project_types=project_types.withColumnRenamed('project_type_id','Mã loại dự án')\
                           .withColumnRenamed('Loại dự án','Tên loại dự án')
    # Sắp xếp DataFrame theo cột "Mã loại dự án" tăng dần
project_types=project_types.orderBy("Mã loại dự án")
                 
#bảng chi tiết dự án
project_details = df.select("project_id", "Tên dự án","project_type_id","Loại dự án","Đường","Vị trí","Diện tích (m²)","Chủ đầu tư","Tổng vốn đầu tư (tỷ đồng)","Mô tả")\
                    .withColumnRenamed('project_id', 'Mã dự án') \
                    .withColumnRenamed("Loại dự án", "Tên loại dự án") \
                    .withColumnRenamed('project_type_id', 'Mã loại dự án')
#bảng vị trí 
locations =df.select('project_id','Vị trí','Đường','Diện tích (m²)')\
            .withColumnRenamed('project_id','Mã dự án')
#bảng trạng thái  
status=df.select('project_id',"Tên dự án",'Trạng thái','Ngày đăng','Xếp hạng','Mô tả')\
         .withColumnRenamed('project_id','Mã dự án')

In [36]:
#thêm khóa ngoại cho bảng project_detail để nối với bảng project
project_details=project_details.join(projects.select(
    'Mã dự án'),on='Mã dự án',how='inner')

#thêm khóa ngoại cho bảng location để nối với bảng project
locations=locations.join(projects.select(
    'Mã dự án'),on='Mã dự án',how='inner')

#thêm khóa ngoại cho bảng status để nối với bảng project
status=status.join(projects.select(
    'Mã dự án'),on='Mã dự án',how='inner') 

#thêm khóa ngoại cho bảng project để nối với bảng project type
projects=projects.join(project_types.select(
    'Mã loại dự án'),on='Mã loại dự án',how='inner')    
       

In [37]:
# Kết nối tới PostgreSQL
jdbc_url = "jdbc:postgresql://localhost:5432/cafeland"
connection_properties = {
    "user": "postgres",
    "password": "123",#nhập mk tạo trong postgres
    "driver": "org.postgresql.Driver"
}

In [38]:
# Hàm ghi dữ liệu vào PostgreSQL

def write_to_postgres(df, table_name):
    try:
        df.write.jdbc(
            url=jdbc_url,
            table=table_name,
            mode="overwrite",
            properties=connection_properties
        )
        print(f"Đã ghi thành công bảng {table_name} vào PostgreSQL!")
    except Exception as e:
        print(f"Lỗi khi ghi bảng {table_name} vào PostgreSQL: {e}")

In [39]:
# Lưu bảng  vào PostgreSQL
write_to_postgres(projects, "projects")
write_to_postgres(locations, "locations")
write_to_postgres(project_details, "project_details")
write_to_postgres(project_types, "project_types")
write_to_postgres(status, "status")

Đã ghi thành công bảng projects vào PostgreSQL!
Đã ghi thành công bảng locations vào PostgreSQL!
Đã ghi thành công bảng project_details vào PostgreSQL!
Đã ghi thành công bảng project_types vào PostgreSQL!
Đã ghi thành công bảng status vào PostgreSQL!


In [40]:
import psycopg2
import logging

# Cấu hình logging
logging.basicConfig(level=logging.INFO)

# Danh sách các lệnh SQL
commands = [
    # Khóa chính
    'ALTER TABLE projects ADD CONSTRAINT pk_project PRIMARY KEY ("Mã dự án");',
    'ALTER TABLE project_types ADD CONSTRAINT pk_type PRIMARY KEY ("Mã loại dự án");',

    # Khóa ngoại
    'ALTER TABLE locations ADD CONSTRAINT fk_locations_projects FOREIGN KEY ("Mã dự án") REFERENCES projects ("Mã dự án");',
    'ALTER TABLE project_details ADD CONSTRAINT fk_project_details_project FOREIGN KEY ("Mã dự án") REFERENCES projects ("Mã dự án");',
    'ALTER TABLE status ADD CONSTRAINT fk_status_project FOREIGN KEY ("Mã dự án") REFERENCES projects ("Mã dự án");',
    'ALTER TABLE projects ADD CONSTRAINT fk_type_project FOREIGN KEY ("Mã loại dự án") REFERENCES project_types ("Mã loại dự án");'
]

def execute_sql_commands(commands):
    try:
        # Kết nối đến PostgreSQL
        connection = psycopg2.connect(
            host="localhost",  # Thay đổi nếu bạn chạy trong Docker
            port="5432",
            database="cafeland",
            user="postgres",
            password="123"
        )
        connection.autocommit = True
        cursor = connection.cursor()

        # Thực thi từng lệnh SQL
        for command in commands:
            cursor.execute(command)
            logging.info(f"Executed command: {command}")

        # Đóng kết nối
        cursor.close()
        connection.close()
        logging.info("Primary and foreign key constraints created successfully.")
    except Exception as e:
        logging.exception(f"Error executing SQL commands: {e}")

# Gọi hàm để thực thi
execute_sql_commands(commands)


INFO:root:Executed command: ALTER TABLE projects ADD CONSTRAINT pk_project PRIMARY KEY ("Mã dự án");
INFO:root:Executed command: ALTER TABLE project_types ADD CONSTRAINT pk_type PRIMARY KEY ("Mã loại dự án");
INFO:root:Executed command: ALTER TABLE locations ADD CONSTRAINT fk_locations_projects FOREIGN KEY ("Mã dự án") REFERENCES projects ("Mã dự án");
INFO:root:Executed command: ALTER TABLE project_details ADD CONSTRAINT fk_project_details_project FOREIGN KEY ("Mã dự án") REFERENCES projects ("Mã dự án");
INFO:root:Executed command: ALTER TABLE status ADD CONSTRAINT fk_status_project FOREIGN KEY ("Mã dự án") REFERENCES projects ("Mã dự án");
INFO:root:Executed command: ALTER TABLE projects ADD CONSTRAINT fk_type_project FOREIGN KEY ("Mã loại dự án") REFERENCES project_types ("Mã loại dự án");
INFO:root:Primary and foreign key constraints created successfully.


In [41]:
# Dừng SparkSession
spark.stop()